### Step 1 상자로서의 변수
- 벡터의 차원 == 벡터의 원소 수
- 배열의 차원 == 축의 개수

    ex) `np.array([1, 2, 3])`은 3차원 벡터이고 1차원 배열이다

In [13]:
class Variable:
    def __init__(self, data):
        self.data = data

In [14]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [15]:
x.data = np.array(2.0)
print(x.data)

2.0


In [16]:
import numpy as np
x = np.array(1)
x.ndim

0

In [17]:
x = np.array([1, 2, 3])
x.ndim

1

In [18]:
x = np.array([[1, 2, 3],
              [4, 5, 6]])
x.ndim

2

### Step 2 변수를 낳는 함수
```
def forward(self, x):
    raise NotImplementedError()
``` 

위와 같이 예외를 발생시키면 해당 메서드는 상속하여 구현해야 한다는 사실을 알려줄 수 있다

In [19]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2
        output = Variable(y)
        return output

In [20]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [21]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self):
        raise NotImplementedError()

In [22]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [23]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


### Step 3 함수 연결

In [24]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [25]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)


1.648721270700128


### Step 4 수치 미분

- 수치 미분(numerical differentiation): 미세한 차이를 이용하여 함수의 변화량을 구하는 방법
- 중앙 차분(centered difference): $\frac{f(x+h) - f(x-h)}{2h}$ -> (수치 미분은 진정한 미분을 근사하는데) 근사 오차를 줄일 수 있는 방법
- 전진 차분(forward difference): $\frac{f(x+h) - f(x)}{h}$

- 전진 차분보다 중앙 차분이 진정한 미분값에 가깝다 (증명: 테일러 급수 이용)

##### 수치 미분의 문제점
1. 결과에 오차가 포함된다
- 이유: 자릿수 누락 때문
- '차이'를 구하는 계산은 주로 크기가 비슷한 값을 다루기 때문에 계산 결과에서 자릿수 누락이 생겨 유효 자릿수가 줄어들 수 있다

2. 계산량이 많다
- 이유: 변수가 여러 개인 계산을 미분할 경우 변수 각각을 미분해야 하기 때문

    => 역전파 등장

- **수치 미분은 정확한 값을 얻을 수 있어서 역전파 구현을 검증할 때 사용된다 -> *기울기 확인 (gradient checking)***


In [26]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [27]:
f = Square()
x = Variable(np.array(2.0))
numerical_diff(f, x)

4.000000000004

In [28]:
def f(x):
    '''f(x) = (e^(x^2))^2'''
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))


x = Variable(np.array(0.5))
# f'(x) = 2*(e^(x^2))*e^(x^2)*2x
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


### Step 5 역전파 이론
- 전파되는 데이터는 모두 y의 미분값 ($\frac{dy}{dy}, \frac{dy}{db}, \frac{dy}{da}, \frac{dy}{dx}$)

- 손실함수의 각 매개변수에 대한 미분을 계산해야 함
- 미분값을 출력에서 입력방향으로 전파하면 한 번의 전파만으로 모든 매개변수에 대한 미분을 계산할 수 있음 => 그래서 미분을 역전파하는 방식 사용
- 변수는 '통상값', '미분값' 존재 / 함수는 '통상 계산(순전파)', '미분값을 구하기 위한 계산(역전파)' 존재
- **역전파 시에는 순전파 시 이용한 데이터가 필요함**



### Step 6 수동 역전파
- 기울기(gradient): 다변수(e.g. 벡터, 행렬)에 대한 미분

In [29]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

In [30]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 입력 변수를 기억(보관)한다.
        return output

    def forward(self, x):
        raise NotImplementedError
    
    def backward(self, gy):
        raise NotImplementedError

In [31]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [32]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


In [33]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [34]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


### Step 7 역전파 자동화
- Define-by-Run (동적 계산 그래프): 딥러닝에서 수행하는 계산들을 계산 시점에 '연결'하는 방식

[참고]
- 함수의 순서를 리스트로 저장해두면 나중에 거꾸로 추적하는 식으로 역전파를 자동화할 수 있다
- 그러나 분기가 있는 계산 그래프 혹은 변수가 여러 번 사용되는 복잡한 계산 그래프는 단순 리스트로 저장하는 식으로 풀 수 없다
- 리스트 데이터 구조를 응용하여 수행한 계산을 리스트에 추가해나가면 어떤 계산 그래프의 역전파도 해낼 수 있다 => 이 데이터 구조를 Wengert List 또는 Tape라고 한다.

#### 변수와 함수의 관계?
- 함수에게 변수는 '입력'과 '출력'
- 변수에게 함수는 '창조자' 혹은 '부모'

이 관계를 맺어주기 위해 `Variable`, `Function` 클래스에 각각 서로에 대한 정보를 넣어줌


In [35]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

In [36]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 출력 변수에 창조자를 설정한다.
        self.input = input
        self.output = output # 출력도 저장
        return output

In [37]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


In [38]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [39]:
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져온다.
b = C.input # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다.

In [40]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [41]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [42]:
class Variable:
    def __init__(self, data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [43]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### Step 8 재귀에서 반복문으로
- 반복문으로 바꾸는 이유
    1. 효율성: 재귀는 함수를 재귀적으로 호출할 때마다 중간 결과를 메모리에 유지하면서 (스택에 쌓으면서) 처리를 이어가기 때문에 일반적으로 반복문 방식이 효율이 더 좋다
    
    2. 확장성: 복잡한 계산 그래프를 다룰 때 더 부드럽게 확장 가능

- 꼬리 재귀(tail recursion) 기법을 이용하여 재귀를 반복문처럼 실행할 수 있는 경우도 있다

In [44]:
class Variable:
    def __init__(self, data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [45]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)


3.297442541400256


### Step 9 함수를 더 편리하게

In [46]:
def square(x):
    # f = Square()
    # return f(x)
    return Square()(x)

def exp(x):
    return Exp()(x)


In [47]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(f'x grad: {x.grad}, a grad: {a.grad}, b grad: {b.grad}, y grad: {y.grad}')

x grad: 3.297442541400256, a grad: 3.297442541400256, b grad: 2.568050833375483, y grad: 1.0


##### `np.ones_like(self.data)` : self.data와 형상과 데이터 타입이 같은 ndarray 인스턴스를 생성하고 요소를 1로 채워준다

In [48]:
class Variable:
    def __init__(self, data) -> None:
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [49]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


#### ndarray만 취급

In [50]:
class Variable:
    def __init__(self, data) -> None:
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
                
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [51]:
x = Variable(None)

In [52]:
x = Variable(1)

TypeError: <class 'int'>은(는) 지원하지 않습니다.

In [53]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [54]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self) # 출력 변수에 창조자를 설정한다.
        self.input = input
        self.output = output # 출력도 저장
        return output

### Step 10 테스트
- 테스트 할 때는 이름이 test로 시작하는 메서드를 만들고 그 안에 테스트할 내용을 적는다
- `self.assertEqual`, `self.assertGreater`, `self.assertTrue`

- 테스트 실행:
    - `$ python -m unittest 파일명.py` 
    (파일명은 기본적으로 test*.py 형태, 변경 가능)
    
    - 혹은 파일 끝에 `unittest.main()` 추가 후 실행


- 디렉터리에 들어있는 모든 테스트 한번에 실행
    - `$ python -m unittest discover 디렉터리명`


- 지속적 통합(CI, continuous integration) 서비스: 깃허브 저장소에 이 서비스를 연계하면 코드를 푸시하고, 풀리퀘스트를 병합하고, 매시간 자동으로 테스트 실행되도록 설정할 수 있다

- 깃허브 저장소를 CI (continuous integration) 도구와 연계하면 소스코드를 지속해서 테스트할 수 있다 => 코드의 신뢰성 유지


In [55]:
import unittest
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

---

컴퓨터로 미분을 계산하는 방법 3가지
1. 수치 미분 (numerical differentiation)
2. 기호 미분 (symbolic differentiation)
    - 미분 공식 이용
    - 입력, 출력이 모두 '수식'
3. 자동 미분 (automatic differentiation)
    - 연쇄 법칙(chain rule)을 사용하여 미분하는 방법
    - forward 모드, reverse 모드로 나눌 수 있다
    - 역전파는 reverse 모드 자동 미분

*역전파
- 출력이 하나뿐이고 그 하나의 출력 변수를 미분하려면 reverse 모드 자동미분이 적합
- 머신러닝은 대부분 출력이 변수 하나로 모아지는 문제를 다루기 때문에 reverse 모드 자동 미분이 사용된다